In [ ]:
import pandas as pd
import urllib
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/nick-edu/dmmldl/master/ForestFires.csv')
display(df)

In [ ]:
url = "https://raw.githubusercontent.com/nick-edu/dmmldl/master/ForestFiresColumns.txt"
file = urllib. request. urlopen(url)
expl_text = []
for line in file: 
    decoded_line = line.decode("utf-8")
    expl_text.append(decoded_line)
print(expl_text)

expl_dict = {}
for line in expl_text:
    
    try:
        key = re.search("(^\w+)\:\s(.*)",line).group(1)
        val = re.search("(^\w+)\:\s(.*)",line).group(2)
    except:
        continue
    expl_dict[key]  = val


print(expl_dict)

In [ ]:
plot_col_drop = ['X','Y']
plot_df = (df
    .drop(plot_col_drop,axis=1))

#converting to log, rounding

area_vals = plot_df['area'] != 0
plot_df['area'] = (np
    .log(plot_df['area'] + 1)
    .round(decimals =3))


months = {'month':{"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12}}
plot_df.replace(months,inplace = True)
display(plot_df)


#pairplot
sns.pairplot(plot_df)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import ipympl

%matplotlib widget
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink']


#Select columns
cols = ['DMC', 'RH', 'area']

#assign chosen columns to X 
X = df.loc[:,cols]
X['area'] = np.log(X['area'] + 1)


#create pipeline 
pipe = Pipeline([("minmax_scaler",StandardScaler())])

X_scaled = pipe.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns = cols)
display(X_scaled)


def plot_cluster(data, cluster_labels):
    fig, ax = plt.subplots(1, 1, figsize=(7,6))
    ax.set_xlim(min(data[:,0]), max(data[:,0]))
    ax.set_ylim(min(data[:,1]), max(data[:,1]))
    cluster_labels = pd.DataFrame(cluster_labels)
    colors = {-1:'tab:blue', 0:'tab:orange', 1:'tab:green', 2:'tab:red', 3:'tab:purple', 4:'tab:brown', 5:'tab:pink'}
    ax.scatter(data[:,0], data[:,1], s=20, lw=0, c = cluster_labels[0].map(colors))

    fig.tight_layout()
    plt.show()



kmeans_kwargs = {
     "init": "random",
     "n_init": 10,
     "max_iter": 300,
     "random_state": 42, }

# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(X_scaled)
    sse.append(kmeans.inertia_)
    

plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

kmeans = KMeans(
     init="random",
     n_clusters=5,
     n_init=10,
     max_iter=300,
     random_state=42 )
X_scaled['cluster'] = kmeans.fit_predict(X_scaled)

X_scaled['c'] = X_scaled.cluster.map({0:colors[0], 1:colors[1], 2:colors[2], 3:colors[3], 4:colors[4]})
fig = plt.figure(figsize=(26,6))
ax = fig.add_subplot(131, projection='3d')
ax.scatter(X_scaled.RH, X_scaled.DMC, X_scaled.area, c = X_scaled.c, s=10)
ax.set_xlabel('RH')
ax.set_ylabel('DC')
ax.set_zlabel('temp')
plt.show()

#df_1 = X_scaled[['DMC','area']]
#df_2 = X_scaled[['DMC','RH']]
#df_3 = X_scaled[['RH','area']]
#DBSCAN_1 = DBSCAN(eps=20, min_samples=2).fit(df_1)
#DBSCAN_2 = DBSCAN(eps=20, min_samples=2).fit(df_2)
#DBSCAN_3 = DBSCAN(eps=20, min_samples=5).fit(df_3)

#plot_cluster(df_1.to_numpy(),DBSCAN_1.labels_)
#plot_cluster(df_2.to_numpy(),DBSCAN_2.labels_)
#plot_cluster(df_3.to_numpy(),DBSCAN_3.labels_)

In [ ]:
X_scaled = X_scaled[X_scaled['area'] != 0]
#sns.heatmap(X_scaled.corr().round(decimals=2),annot=True)


df_test = df[df['area'] != 0].replace(months)
df_test['area'] = np.log(df_test['area']+1)
corr_matrix = df_test.corr().round(decimals =2)
print(corr_matrix)
sns.heatmap(df_test.corr().round(decimals = 2), annot = True)

display(df_test)

In [ ]:
def plot_cluster(data, cluster_labels):
    fig, ax = plt.subplots(1, 1, figsize=(7,6))
    ax.set_xlim(min(data[:,0]), max(data[:,0]))
    ax.set_ylim(min(data[:,1]), max(data[:,1]))
    cluster_labels = pd.DataFrame(cluster_labels)
    colors = {-1:'tab:blue', 0:'tab:orange', 1:'tab:green', 2:'tab:red', 3:'tab:purple', 4:'tab:brown', 5:'tab:pink'}
    ax.scatter(data[:,0], data[:,1], s=20, lw=0, c = cluster_labels[0].map(colors))

    fig.tight_layout()
    plt.show()

df_1 = X
